In [ ]:
import os
import sys
import time
import errno
import time
import timeit
import pickle
import gc
import re
import glob
from tqdm.notebook import trange, tqdm
from typing import Any, Callable, Dict, List, Optional, Tuple, Union, cast

import numpy as np
import scipy as scp
from scipy import interpolate
from scipy import signal
from scipy.stats.mstats import mquantiles
from scipy.ndimage import gaussian_filter1d
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import xarray as xr
from sklearn import preprocessing

from scipy import stats
from scipy.stats import mode
from scipy.signal import argrelextrema, argrelmax, argrelmin
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker


from scipy import signal
from scipy.signal import find_peaks, peak_widths, savgol_filter
from scipy import stats as spstats
from scipy.stats import kurtosis, skew, mode, moment
from scipy.signal import hilbert
from scipy.optimize import curve_fit
from sklearn.preprocessing import minmax_scale

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action="ignore", category=FutureWarning)
np.seterr(all = 'ignore')

from matplotlib import colors, cm
import matplotlib.colors as mcolors
from IPython.display import HTML, display, clear_output
from statannotations.Annotator import Annotator

import numba
from numba import jit
import multiprocessing as mp

import torch
import sbi 
import sbi.inference
from sbi.inference.base import infer
from sbi.inference import SNPE, SNLE, SNRE, prepare_for_sbi ,simulate_for_sbi
from sbi.inference import likelihood_estimator_based_potential, DirectPosterior, MCMCPosterior, VIPosterior
from sbi.analysis import ActiveSubspace, pairplot
import sbi.utils as utils

import nest 
import nest.raster_plot
nest.set_verbosity(100)
import pylab as pl 

In [ ]:
import glob
import os
import sys
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from numba import jit
import pandas as pd
from scipy.io import loadmat
from scipy import signal

In [ ]:
#####################################################
def LSE(x1, x2):
    return np.sum((x1 - x2)**2)
#####################################################
def Err(x1, x2):
    return np.sum(np.abs(x1 - x2))
#####################################################    
def RMSE(x1, x2):
    return np.sqrt(((x1 - x2) ** 2).mean()) 
#####################################################
def LSE_obs(Obs, Obs_lo, Obs_hi):
    return np.average([LSE(Obs, Obs_lo), LSE(Obs, Obs_hi)])
#####################################################
def z_score(true_mean, post_mean, post_std):
    return np.abs((post_mean - true_mean) / post_std)
#####################################################
def shrinkage(prior_std, post_std):
    return 1 - (post_std / prior_std)**2
#####################################################
import torch
import numpy as np
from scipy.stats import gaussian_kde
from sbi.analysis.plot import _get_default_opts, _update, ensure_numpy

def _get_limits(samples, limits=None):

    if type(samples) != list:
        samples = ensure_numpy(samples)
        samples = [samples]
    else:
        for i, sample_pack in enumerate(samples):
            samples[i] = ensure_numpy(samples[i])

    # Dimensionality of the problem.
    dim = samples[0].shape[1]

    if limits == [] or limits is None:
        limits = []
        for d in range(dim):
            min = +np.inf
            max = -np.inf
            for sample in samples:
                min_ = sample[:, d].min()
                min = min_ if min_ < min else min
                max_ = sample[:, d].max()
                max = max_ if max_ > max else max
            limits.append([min, max])
    else:
        if len(limits) == 1:
            limits = [limits[0] for _ in range(dim)]
        else:
            limits = limits
    limits = torch.as_tensor(limits)

    return limits

def posterior_peaks(samples, return_dict=False, **kwargs):
    '''
    Finds the peaks of the posterior distribution.

    Args:
        samples: torch.tensor, samples from posterior
    Returns: torch.tensor, peaks of the posterior distribution
            if labels provided as a list of strings, and return_dict is True
            returns a dictionary of peaks

    '''

    opts = _get_default_opts()
    opts = _update(opts, kwargs)

    limits = _get_limits(samples)
    samples = samples.numpy()
    n, dim = samples.shape

    try:
        labels = opts['labels']
    except:
        labels = range(dim)

    peaks = {}
    if labels is None:
        labels = range(dim)
    for i in range(dim):
        peaks[labels[i]] = 0

    for row in range(dim):
        density = gaussian_kde(
            samples[:, row],
            bw_method=opts["kde_diag"]["bw_method"])
        xs = np.linspace(
            limits[row, 0], limits[row, 1],
            opts["kde_diag"]["bins"])
        ys = density(xs)

        # y, x = np.histogram(samples[:, row], bins=bins)
        peaks[labels[row]] = xs[ys.argmax()]

    if return_dict:
        return peaks
    else:
        return list(peaks.values())
    
def plot_posterior(samples,
                   ax,
                   prob=[0.025, 0.975],
                   labels=None,
                   xlim=None,
                   ylim=None,
                   xticks=None,
                   yticks=None,
                   xlabel=None,
                   ylabel=None):

    if ax is None:
        print('pass axis!')
        exit(0)

    samples = samples.numpy()
    # print(type(samples))
    # print(samples.shape)
    dim = samples.shape[1]
    # assert (len(ax) == dim)
    if labels is not None:
        assert (len(labels) == dim)

    hist_diag = {"alpha": 1.0, "bins": 50, "density": True, "histtype": "step"}

    max_values = np.zeros(dim)

    for i in range(dim):
        ax0 = ax if (dim == 1) else ax[i]
        n, bins, _ = ax0.hist(samples[:, i], **hist_diag)

        if labels is not None:
            ax0.set_xlabel(labels[i], fontsize=13)
        ax0.tick_params(labelsize=12)

        xs = mquantiles(samples[:, i], prob)

        max_value = bins[np.argmax(n)]
        max_values[i] = max_value
        ax0.axvline(x=max_value, ls='--', color='gray', lw=2)
        # for j in range(2):
        #     ax0.axvline(x=xs[j], ls='--', color="royalblue", lw=2)
        y = n[np.where((bins > xs[0]) & (bins < xs[1]))]

        ax0.fill_between(np.linspace(xs[0], xs[1], len(y)), y,
                         color="gray",
                         alpha=0.2)

        ax0.set_title("{:g}".format(max_value))

    plt.tight_layout()

    return list(max_values)

In [ ]:
###############################################################################
# Definition of functions used in this example. First, define the `Lambert W`
# function implemented in SLI. The second function computes the maximum of
# the postsynaptic potential for a synaptic input current of unit amplitude
# (1 pA) using the `Lambert W` function. Thus function will later be used to
# calibrate the synaptic weights.

def LambertWm1(x):
    # Using scipy to mimic the gsl_sf_lambert_Wm1 function.
    return scp.special.lambertw(x, k=-1 if x < 0 else 0).real


def ComputePSPnorm(tauMem, CMem, tauSyn):
    a = (tauMem / tauSyn)
    b = (1.0 / tauSyn - 1.0 / tauMem)

    # time of maximum
    t_max = 1.0 / b * (-LambertWm1(-np.exp(-1.0 / a) / a) - 1.0 / a)

    # maximum of PSP for current of unit amplitude
    return (np.exp(1.0) / (tauSyn * CMem * b) *
            ((np.exp(-t_max / tauMem) - np.exp(-t_max / tauSyn)) / b -
             t_max * np.exp(-t_max / tauSyn)))

In [ ]:
# Function to trial-separate neural timestamps
def formatData_Spike(data, time, timeLocks, binSize, maxLead, maxLag, method):
    
    if len(data) != len(time):
        raise Exception('Data and time must have equal length')

    binBounds = np.linspace(-np.round(maxLead/binSize, 1),np.round(maxLag/binSize, 1), int((maxLead+maxLag)/binSize)+1)*binSize

    timeBounds = np.array(list(zip(binBounds[0:-1],binBounds[1:])))

    nTrials = len(timeLocks)
    nBins = len(timeBounds)

    fData = np.zeros((nBins, nTrials))

    for iTrial in range(nTrials):

        for iBin in range(nBins):
            if method == 'count':
                fData[iBin, iTrial] = np.sum(data[(time >= (timeLocks[iTrial] + timeBounds[iBin][0])) & (time < (timeLocks[iTrial] + timeBounds[iBin][1]))])
            elif method == 'rate':
                spkCount = np.sum(data[(time >= (timeLocks[iTrial] + timeBounds[iBin][0])) & (time < (timeLocks[iTrial] + timeBounds[iBin][1]))])     
                fData[iBin, iTrial] = spkCount/np.diff(timeBounds[iBin,:])
                
    return fData, timeBounds

In [ ]:
def spike_to_rate(spiketimes, nbins = 100, remove_tails = False):

    spiketimes = spiketimes[np.array(spiketimes)!=None]

    binned_spikes, _ = np.histogram(spiketimes, bins = nbins, range = (0,1000))

    binned_fr = smooth_rates(binned_spikes, nbins = nbins, remove_tails = remove_tails)
    
    return binned_fr

spike_to_rate = jit(spike_to_rate)

def smooth_rates(firing_rate, nbins = 100, remove_tails = False, lp_filtfilt=3, lp_savgol=2, nneigh=70, savgol_order=5):
        
    if remove_tails:
        lowpass = signal.butter(5, 3, 'lp', fs=nbins, output='sos')
        firing_rate = savgol_filter(firing_rate, nneigh, savgol_order, mode = 'mirror')
    else:
        lowpass = signal.butter(5, 2, 'lp', fs=nbins, output='sos')

    firing_rate = signal.sosfiltfilt(lowpass, firing_rate, axis = 0)
    
    return firing_rate

In [ ]:
def modify_axis_spines(ax, which=None, base=1.0, xticks=[], yticks=[], yaxis_left=True, xaxis_bot=True):

    tick_locator = plticker.MultipleLocator(base=base)

    if yaxis_left: 
        ax.spines.right.set(visible=False)
        yspine = ax.spines.left
    else:
        ax.spines.left.set(visible=False)
        yspine = ax.spines.right
        
    if xaxis_bot:
        ax.spines.top.set(visible=False)
        xspine = ax.spines.bottom
    else:
        ax.spines.bottom.set(visible=False)
        xspine = ax.spines.top
                           
    if 'x' in which:
        if len(xticks) == 0:
            xticks = ax.get_xticks() 
            ax.xaxis.set_major_locator(tick_locator)
        ax.set_xticks(xticks)
        xspine.set_bounds(ax.get_xticks()[0], ax.get_xticks()[-1])
        
    else:
        ax.spines.bottom.set(visible=False)
    
    if 'y' in which:
        if len(yticks) == 0:
            yticks = ax.get_yticks()
        ax.set_yticks(yticks)
        yspine.set_bounds(ax.get_yticks()[0], ax.get_yticks()[-1])
        if len(yticks) == 0:
            ax.yaxis.set_major_locator(tick_locator)
    else:
        ax.spines.left.set(visible=False)

def fmt_plot_text(text):
    return f'{text:.2f}'

def get_source_time_label(time_value):
    return 'Time from ball movement: {} ms'.format(np.round(time_value*1000))

def plot_spk_avg_rates(fig, ax, firing_rates_emp, shade_colors=['cornflowerblue', 'lightcoral'], alpha=0.5, baseline=False):

    p_rates = firing_rates_emp.sel(condition='Presence').copy()
    a_rates = firing_rates_emp.sel(condition='Absence').copy()
    
    p_rate_avg =  p_rates.mean('neuron')
    a_rate_avg =  a_rates.mean('neuron')
    
    p_rate_err = stats.sem(p_rates.to_numpy().T)
    a_rate_err = stats.sem(a_rates.to_numpy().T)
    
    ax.plot(p_rates.mean('neuron'), label='Presence', c='dodgerblue');
    ax.plot(a_rates.mean('neuron'), label='Absence', c='crimson')
    
    ax.fill_between(time_vec, p_rate_avg - p_rate_err, p_rate_avg + p_rate_err, color='cornflowerblue', alpha=0.5, linewidth=0)
    ax.fill_between(time_vec, a_rate_avg - a_rate_err, a_rate_avg + a_rate_err, color='lightcoral', alpha=0.5, linewidth=0)
    
    ax.set_ylabel('Average firing rate')
    ax.set_xlabel('Time (ms)')
    
    modify_axis_spines(ax, which = ['x', 'y'], xticks = np.arange(0, 101, 50), yticks = np.arange(np.min([p_rate_avg, a_rate_avg]), np.max([p_rate_avg, a_rate_avg])+0.1, 0.3)) 
    ax.set_xticklabels(['-500', 'FB', '500'])
    
    lgd = ax.legend(frameon=False)
    
fig.savefig(fig_save_loc + save_string + 'AllNeurons_averageFiringRate_STE_nFB.svg', transparent = True, bbox_inches = 'tight')

In [ ]:
def scale_sim_to_emp(sim_data, emp_data):

    emp_range = emp_data.max() - emp_data.min()
    sim_range = sim_data.max() - sim_data.min()

    range_ratio = emp_range/sim_range

    sim_data *= range_ratio

    start_diff = emp_data[0] - sim_data[0]
    sim_data += start_diff

    return sim_data

# Data directories

In [ ]:
fig_save_loc = ''
save_string = 'SPK/'
parent_preprocess_dir = ''

# Brunel model

In [ ]:
###############################################################################
nest.ResetKernel()
n = 8  # number of threads

nest.SetKernelStatus({'local_num_threads': n, 'print_time': False})

startbuild = time.time()

dt = 0.1    # the resolution in ms
simtime = 1000.0  # Simulation time in ms
delay = 1.5    # synaptic delay in ms

g = 5.8  # ratio inhibitory weight/excitatory weight
eta = 1.5  # external rate relative to threshold rate
epsilon = 0.1  # connection probability

order = 2500
NE = 4 * order  # number of excitatory neurons
NI = 1 * order  # number of inhibitory neurons
N_neurons = NE + NI   # number of neurons in total
N_rec = 100      # record from 50 neurons

CE = int(epsilon * NE)  # number of excitatory synapses per neuron
CI = int(epsilon * NI)  # number of inhibitory synapses per neuron
C_tot = int(CI + CE)      # total number of synapses per neuron

tauSyn = 0.5  # synaptic time constant in ms
tauMem = 20.0  # time constant of membrane potential in ms
CMem = 250.0  # capacitance of membrane in in pF
theta = 20.0  # membrane threshold potential in mV

neuron_params = {"C_m": CMem,
                 "tau_m": tauMem,
                 "tau_syn_ex": tauSyn,
                 "tau_syn_in": tauSyn,
                 "t_ref": 2.0,
                 "E_L": 0.0,
                 "V_reset": 10.0,
                 "V_m": 0.0,
                 "V_th": theta}

J = 0.1        # postsynaptic amplitude in mV
J_unit = ComputePSPnorm(tauMem, CMem, tauSyn)
J_ex = J / J_unit  # amplitude of excitatory postsynaptic current
J_in = -g * J_ex    # amplitude of inhibitory postsynaptic current

nu_th = (theta * CMem) / (J_ex * CE * np.exp(1) * tauMem * tauSyn)
nu_ex = eta * nu_th
p_rate = 1000.0 * nu_ex * CE
# p_rate = 9000

nest.resolution = dt
nest.print_time = True
nest.overwrite_files = True

nodes_ex = nest.Create("iaf_psc_alpha", NE, params=neuron_params)
nodes_in = nest.Create("iaf_psc_alpha", NI, params=neuron_params)
noise = nest.Create("poisson_generator", params={"rate": p_rate})
espikes = nest.Create("spike_recorder")
ispikes = nest.Create("spike_recorder")


###############################################################################
# Set external current

pop_stepcurrent = nest.Create('noise_generator', n = 1)

time_start = 350
time_end = 900
meanCurr = 150
stdCurr = 1

pop_stepcurrent.set(start = time_start, stop = time_end, mean = meanCurr, std = stdCurr)
nest.Connect(pop_stepcurrent, nodes_ex, syn_spec={'weight': 1.})
pop_stepcurrent.set(start = time_start, stop = time_end, mean = meanCurr, std = stdCurr)
nest.Connect(pop_stepcurrent, nodes_in, syn_spec={'weight': 1.})

###############################################################################

espikes.set(label="brunel-py|-ex", record_to="memory")
ispikes.set(label="brunel-py-in", record_to="memory")

nest.CopyModel("static_synapse", "excitatory",
               {"weight": J_ex, "delay": delay})
nest.CopyModel("static_synapse", "inhibitory",
               {"weight": J_in, "delay": delay})

nest.Connect(noise, nodes_ex, syn_spec="excitatory")
nest.Connect(noise, nodes_in, syn_spec="excitatory")

nest.Connect(nodes_ex[:N_rec], espikes, syn_spec="excitatory")
nest.Connect(nodes_in[:N_rec], ispikes, syn_spec="excitatory")

conn_params_ex = {'rule': 'fixed_indegree', 'indegree': CE}
nest.Connect(nodes_ex, nodes_ex + nodes_in, conn_params_ex, "excitatory")

conn_params_in = {'rule': 'fixed_indegree', 'indegree': CI}
nest.Connect(nodes_in, nodes_ex + nodes_in, conn_params_in, "inhibitory")

endbuild = time.time()

nest.Simulate(simtime)

endsimulate = time.time()

events_ex = espikes.n_events
events_in = ispikes.n_events

rate_ex = events_ex / simtime * 1000.0 / N_rec
rate_in = events_in / simtime * 1000.0 / N_rec

num_synapses_ex = nest.GetDefaults("excitatory")["num_connections"]
num_synapses_in = nest.GetDefaults("inhibitory")["num_connections"]
num_synapses = num_synapses_ex + num_synapses_in

ex_senders = espikes.events['senders']
in_senders = ispikes.events['senders']
ex_times = espikes.events['times']
in_times = ispikes.events['times']

# np.savez_compressed(file = 'tempSim.npz',g = g, ex_senders = ex_senders, in_senders = in_senders, ex_times = ex_times, in_times = in_times)

nest.raster_plot.from_device(espikes, hist=True, hist_binwidth=30.0)
# plt.savefig(fig_save_loc + 'Brunel_Nest_testsim.png', dpi = 400, transparent = True)
plt.title('Sample brunel simulation')
# plt.savefig(fig_save_loc + save_string + 'Brunel_Nest_testsim.png', dpi = 400, transparent = True)
plt.show()
print(endsimulate - endbuild)


In [ ]:
def simulate_brunel(g, save_name = '', save_path = '', save_results = False):
    
    ###############################################################################
    # Setting up simulation parameters
    
    g = np.array(g).ravel()[0]
    
    dt = 0.1    # the resolution in ms
    simtime = 1000.0  # Simulation time in ms    
    num_workers = 8  # number of threads
    
    nest.ResetKernel()
    nest.SetKernelStatus({'local_num_threads': num_workers, "print_time": False, "overwrite_files": True})
    nest.resolution = dt

    ###############################################################################
    # Neural parameters
    
    ctime_start = 350 # start of external current
    ctime_end = 900 # end of external current
    meanCurr = 150
    stdCurr = 1
    
    delay = 1.5    # synaptic delay in ms
    eta = 1  # external rate relative to threshold rate
    tauSyn = 0.5  # synaptic time constant in ms
    tauMem = 20.0  # time constant of membrane potential in ms
    CMem = 250.0  # capacitance of membrane in in pF
    theta = 20.0  # membrane threshold potential in mV
    
    neuron_params = {"C_m": CMem,
                     "tau_m": tauMem,
                     "tau_syn_ex": tauSyn,
                     "tau_syn_in": tauSyn,
                     "t_ref": 2.0,
                     "E_L": 0.0,
                     "V_reset": 10.0,
                     "V_m": 0.0,
                     "V_th": theta}
    
    
    J = 0.1        # postsynaptic amplitude in mV
    J_unit = ComputePSPnorm(tauMem, CMem, tauSyn)
    J_ex = J / J_unit  # amplitude of excitatory postsynaptic current
    J_in = -g * J_ex    # amplitude of inhibitory postsynaptic current
        
    ###############################################################################
    # Network parameters
    
    order = 2500
    NE = 4 * order  # number of excitatory neurons
    NI = 1 * order  # number of inhibitory neurons
    N_neurons = NE + NI   # number of neurons in total
    epsilon = 0.1  # connection probability
    N_rec = 100      # record from N_rec neurons

    CE = int(epsilon * NE)  # number of excitatory synapses per neuron
    CI = int(epsilon * NI)  # number of inhibitory synapses per neuron
    C_tot = int(CI + CE)      # total number of synapses per neuron
    
    conn_params_ex = {'rule': 'fixed_indegree', 'indegree': CE}
    conn_params_in = {'rule': 'fixed_indegree', 'indegree': CI}
    
    ###############################################################################
    # Rate of external poisson input
    
    nu_th = (theta * CMem) / (J_ex * CE * np.exp(1) * tauMem * tauSyn)
    nu_ex = eta * nu_th
    p_rate = 1000.0 * nu_ex * CE
    
    ###############################################################################
    # Set recorders and stimulator
    
    nodes_ex = nest.Create("iaf_psc_alpha", NE, params=neuron_params)
    nodes_in = nest.Create("iaf_psc_alpha", NI, params=neuron_params)
    noise = nest.Create("poisson_generator", params={"rate": p_rate})
    espikes = nest.Create("spike_recorder")
    ispikes = nest.Create("spike_recorder")
    pop_stepcurrent = nest.Create('noise_generator', n = 1)

    ###############################################################################
    # Set recorders and stimulator
    
    pop_stepcurrent.set(start = ctime_start, stop = ctime_end, mean = meanCurr, std = stdCurr)
    pop_stepcurrent.set(start = ctime_start, stop = ctime_end, mean = meanCurr, std = stdCurr)
    espikes.set(label="brunel-ex", record_to="memory")
    ispikes.set(label="brunel-in", record_to="memory")

    nest.CopyModel("static_synapse", "excitatory",
                   {"weight": J_ex, "delay": delay})
    nest.CopyModel("static_synapse", "inhibitory",
                   {"weight": J_in, "delay": delay})

    ###############################################################################
    # Connect network elements
    
    nest.Connect(pop_stepcurrent, nodes_ex, syn_spec={'weight': 1.})
    nest.Connect(pop_stepcurrent, nodes_in, syn_spec={'weight': 1.})

    nest.Connect(noise, nodes_ex, syn_spec="excitatory")
    nest.Connect(noise, nodes_in, syn_spec="excitatory")

    nest.Connect(nodes_ex[:N_rec], espikes, syn_spec="excitatory")
    nest.Connect(nodes_in[:N_rec], ispikes, syn_spec="excitatory")
    
    nest.Connect(nodes_ex, nodes_ex + nodes_in, conn_params_ex, "excitatory")
    nest.Connect(nodes_in, nodes_ex + nodes_in, conn_params_in, "inhibitory")
    
    nest.Simulate(simtime)

    ###############################################################################
    # Save spike times to disk
    ex_senders = espikes.events['senders']
    in_senders = ispikes.events['senders']
    ex_times = espikes.events['times']
    in_times = ispikes.events['times']

    if not save_path.endswith('/'):
        save_path = save_path + '/'
    
    if save_results:
        np.savez_compressed(file = save_path + save_name + '.npz', g=g, ex_senders=ex_senders, in_senders=in_senders, ex_times=ex_times, in_times=in_times)
    else:
        return ex_senders, ex_times

## Simulation

In [ ]:
num_sims = 10000

spk_sbi_dir ='./DCM_SPK_SBI/' 
save_name = 'brunel_delta_' + str(num_sims) + '_sims_'
save_path =  spk_sbi_dir + 'input/' + save_name[:-1] + '/simulations/'

In [ ]:
# Prior over model parameters
prior_min = [5.]
prior_max = [8.]
prior = utils.torchutils.BoxUniform(low=torch.as_tensor(prior_min),
                                    high=torch.as_tensor(prior_max))

theta = prior.sample((num_sims,))

In [ ]:
stime = time.time()

for ind, sim_num in enumerate(trange(1,num_sims+1)):
    
    simulate_brunel(theta[ind], save_path = save_path , save_name = save_name + str(sim_num), save_results = True)
    
etime = time.time()
print('Total time: {}'.format(etime - stime))

# Concatenating simulations

In [ ]:
theta = []

ex_senders_agg = []
in_senders_agg = []
ex_times_agg = []
in_times_agg = []

for ind, sim_num in enumerate(trange(1,num_sims+1)):
    
    tempsim = np.load(file = save_path + save_name + str(sim_num) + '.npz')
    theta.append(tempsim['g'])
    ex_senders_agg.append(tempsim['ex_senders'])
    in_senders_agg.append(tempsim['in_senders'])
    ex_times_agg.append(tempsim['ex_times'])
    in_times_agg.append(tempsim['in_times'])

theta = np.array(theta)

In [ ]:
ex_lens = np.array([len(senders) for senders in ex_senders_agg])
in_lens = np.array([len(senders) for senders in in_senders_agg])

max_ex_len = np.max(ex_lens)
max_in_len = np.max(in_lens)

In [ ]:
dims = ('data', 'sim', 'source')
coords = {'source': ['neuron_ind', 'spike_time']}

ex_array = np.empty((max_ex_len, num_sims, 2), dtype=object)
in_array = np.empty((max_in_len, num_sims, 2), dtype=object)

ex_array = xr.DataArray(ex_array, dims=dims, coords=coords, attrs={'Origin': 'Excitatory cells'})
in_array = xr.DataArray(in_array, dims=dims, coords=coords, attrs={'Origin': 'Inhibitory cells'})

In [ ]:
for sInd in trange(num_sims):
    
    ex_array[:ex_lens[sInd], sInd, 0] = ex_senders_agg[sInd]
    ex_array[:ex_lens[sInd], sInd, 1] = ex_times_agg[sInd]
    
    in_array[:in_lens[sInd], sInd, 0] = in_senders_agg[sInd]
    in_array[:in_lens[sInd], sInd, 1] = in_times_agg[sInd]

# Simulation firing rates

In [ ]:
sInd = 4316
sample_times = ex_array[:,sInd,1]
sample_ids = ex_array[:,sInd,0]

sample_times = sample_times[np.array(sample_times)!=None]
sample_ids = sample_ids[np.array(sample_ids)!=None]

nbins = 100
sample_fr, _ = np.histogram(sample_times, bins=nbins, range=(0,1000))

In [ ]:
lowpass = signal.butter(5, 2, 'lp', fs=nbins, output='sos')

new_fr = smooth_rates(sample_fr, remove_tails=True)
plt.plot(sample_fr);plt.plot(new_fr)

In [ ]:
firing_rates_sim = np.zeros((sample_fr.shape[0], num_sims))

for sim_num in trange(num_sims):
    
    spike_times = ex_array.sel(source = 'spike_time', sim = sim_num)
    firing_rate = spike_to_rate(spike_times, remove_tails = True)
    firing_rates_sim[:, sim_num] = firing_rate

## Feature calculation

In [ ]:
spk_sims_path = spk_sbi_dir + 'input/' + save_name[:-1]

if not os.path.isdir(spk_sims_path):
    os.mkdir(spk_sims_path)

In [ ]:
feature_names = ['max', 'argmax', 'mean']
# sel_features = feature_names
sel_features = ['max']


firing_rates_sim_norm = firing_rates_sim.copy()
firing_rates_sim_norm /= firing_rates_sim_norm.max()
firing_rates_sim_norm += 1
firing_rates_sim_norm /= 2


fr_maxs_sim = firing_rates_sim.max(0)[...,None]
fr_avgs_sim = firing_rates_sim.mean(0)[...,None]
fr_argmaxs_sim = firing_rates_sim.argmax(0)[...,None]

normalize=True
if normalize:
    fr_maxs_sim = preprocessing.MinMaxScaler().fit_transform(fr_maxs_sim.reshape(-1,1))
    fr_avgs_sim = preprocessing.MinMaxScaler().fit_transform(fr_avgs_sim.reshape(-1,1))

fr_summary_sim = np.concatenate((fr_maxs_sim, fr_argmaxs_sim, fr_avgs_sim), axis=1)
fr_summary_sim = xr.DataArray(fr_summary_sim, dims=('sample', 'feature'), coords=dict(feature=feature_names))

x_feature_array = fr_summary_sim.copy()
x_features = fr_summary_sim.copy().sel(feature=sel_features)

num_features = len(x_feature_array.feature)

In [ ]:
firing_rates_sim_norm = firing_rates_sim.copy()

fr_maxs = firing_rates_sim_norm.max(0)
fr_maxs = preprocessing.MinMaxScaler().fit_transform(fr_maxs.reshape(-1,1))

plt.scatter(theta, fr_maxs)

In [ ]:
np.savez_compressed(file=spk_sims_path + '/input_data.npz', theta=theta, firing_rates=firing_rates_sim)
x_features.to_netcdf(spk_sims_path + '/features.nc')

## Loading features

In [ ]:
data_dict = np.load(spk_sims_path + '/input_data.npz')
theta = data_dict['theta']
firing_rates_sim = data_dict['firing_rates']

x_feature_array = xr.load_dataarray(spk_sims_path + '/features.nc')
sel_features = ['max']
feature_string = '-'.join(sel_features)

In [ ]:
x_features = np.array(x_feature_array.to_numpy(), dtype='float32')
x_features = torch.as_tensor(x_features)

theta = np.array(theta, dtype='float32')
theta = theta.reshape(theta.shape[0],1)
theta = torch.as_tensor(theta)

In [ ]:
print( 'Theta shape:',theta.shape,flush=True)
print('Feature shape:', x_features.shape,flush=True)
print('Features: ' + feature_string)

In [ ]:
plt.plot(firing_rates_sim, 'k', alpha=0.01);

In [ ]:
plt.scatter(theta, x_features[:,0])

# Inference

In [ ]:
estimator_type = 'nsf'

posterior_string = save_name + feature_string + '_' + estimator_type
print(posterior_string)

In [ ]:
start_time = time.time()

inference = SNPE(prior, density_estimator=estimator_type, device='cpu')

posterior_estimator = inference.append_simulations(theta, x_features).train()

print ("-"*60)
print("---training took:  %s seconds ---" % (time.time() - start_time))

with open(spk_sbi_dir + 'output/estimators/'  + posterior_string + '.pkl', 'wb') as f:
    pickle.dump(posterior_estimator,f)

In [ ]:
posterior_file = open(spk_sbi_dir + 'output/estimators/'  + posterior_string + '.pkl', "rb")
posterior_estimator = pickle.load(posterior_file)
posterior_file.close()

# Empirical data

In [ ]:
firing_rates_emp = xr.load_dataarray(parent_preprocess_dir + 'Demolliens_SPK_rates_norm.nc')
neuron_names = firing_rates_emp.neuron.to_numpy()
time_vec = firing_rates_emp.time.to_numpy()

firing_rates_emp_stacked = firing_rates_emp.stack(condSamples=('condition', 'neuron'))
condition_array = firing_rates_emp_stacked.condition.to_numpy()

In [ ]:
fig, ax = plt.subplots()
plot_spk_avg_rates(fig, ax, firing_rates_emp)

In [ ]:
fr_maxs_emp = firing_rates_emp_stacked.max('time').to_numpy()[...,None]
fr_argmaxs_emp = firing_rates_emp_stacked.argmax('time').to_numpy()[...,None]
fr_avgs_emp = firing_rates_emp_stacked.mean('time').to_numpy()[...,None]

fr_summary_emp = np.concatenate((fr_maxs_emp, fr_argmaxs_emp, fr_avgs_emp), axis=1)
fr_summary_emp = xr.DataArray(fr_summary_emp, dims=('sample', 'feature'), coords=dict(feature=feature_names))

emp_features = fr_summary_emp.copy().sel(feature=sel_features)
num_emp_samples = firing_rates_emp_stacked.condSamples.shape[0]

# Posterior Sampling

In [ ]:
num_samples=1000
posterior = DirectPosterior(posterior_estimator, prior)

In [ ]:
theta_posterior = np.zeros((num_samples, num_emp_samples))

for s_ind in range(num_emp_samples):
    display(f"Iteration: {s_ind}")
    theta_posterior[:, s_ind] = posterior.sample((num_samples,), emp_features.isel(sample=s_ind).to_numpy()).numpy().squeeze()    
    clear_output()

posteriorMaxs = np.array(posterior_peaks(torch.as_tensor(theta_posterior), return_dict=False))

In [ ]:
theta_posterior_array = xr.DataArray(theta_posterior, dims=('sample', 'condition'), coords=dict(condition=condition_array))
theta_posterior_stacked = theta_posterior_array.stack(aggSample=('sample', 'condition'))

posterior_df = theta_posterior_stacked.to_pandas().reset_index().rename(columns={0: 'value'})

In [ ]:
posterior_df

In [ ]:
x = 'condition'
y = 'value'
palette = {'Presence': 'dodgerblue', 'Absence': 'crimson'}

bw = 0.1
width_viol = 0.5
orient = 'v'
alpha = .99
dodge = True
pointplot = False
move = 0.2
point_size = 0
cut = 0.5
scale = 'area'
width_box = 0.5
line_width = 0.1
saturation = 1

In [ ]:
labels = [r'$g$']
lw=1

fig, ax = plt.subplots(constrained_layout=True)

violins = sns.boxplot(x=x, y=y, data=posterior_df, fliersize=0,
               palette=palette, saturation = 0.8, dodge=True, ax=ax, linewidth = 2)

annotator = Annotator(x=x, y=y, ax=ax, pairs=[('Presence','Absence')], data=posterior_df,)
annotator.configure(test='Mann-Whitney', text_format='star', loc='outside', line_width=lw)
annotator.apply_and_annotate()

ax.tick_params(axis='both', which='major', length=0)
ax.set_xlabel('', labelpad = 12)
ax.set_ylabel(r'$g$', labelpad = 12)
ax.set_ylim([5.5, 6.1])
ax.set_frame_on(False)


In [ ]:
posteriorMaxs = xr.DataArray(posteriorMaxs, dims=('sample_2'), coords=dict(sample_2=condition_array))
posterior_array = xr.DataArray(posterior_df['value'].to_numpy(), dims=('sample_1'), coords=dict(sample_1=posterior_df['condition'].to_numpy()))

In [ ]:
posterior_data = xr.Dataset(dict(posterior_array=posterior_array, posterior_maxs=posteriorMaxs))
posterior_data.to_netcdf(parent_preprocess_dir + 'Demolliens_SPK_AllNeurons_Posterior.nc')

# Average firing rates

In [ ]:
presence_mean = firing_rates_emp.sel(condition='Presence').copy().mean('neuron')
absence_mean = firing_rates_emp.sel(condition='Absence').copy().mean('neuron')

mean_rates = np.array([presence_mean, absence_mean]).T

plt.plot(mean_rates)

In [ ]:
fr_avg_maxs = mean_rates.mean(0)[...,None]
fr_avg_argmax = mean_rates.argmax(0)[...,None]
fr_avg_means = mean_rates.mean(0)[...,None]

fr_avg_summary = np.concatenate((fr_avg_maxs, fr_avg_argmax, fr_avg_means), axis=1)
fr_avg_summary = xr.DataArray(fr_avg_summary, dims=('sample', 'feature'), coords=dict(feature=feature_names))

emp_features = fr_avg_summary.copy().sel(feature=sel_features)
num_emp_samples = emp_features.sample.shape[0]

In [ ]:
theta_posterior_avg = np.array([posterior.sample((num_samples,), emp_features.isel(sample=s_ind).to_numpy()).numpy() for s_ind in range(num_emp_samples)]).squeeze().T
posteriorMaxs_avg = np.array(posterior_peaks(torch.as_tensor(theta_posterior_avg), return_dict=False))

In [ ]:
theta_posterior_avg = xr.DataArray(theta_posterior_avg, dims=('sample', 'condition'), coords=dict(condition=['Presence', 'Absence']))
theta_posterior_stacked = theta_posterior_avg.stack(aggSample=('sample', 'condition'))

posterior_df_avg = theta_posterior_stacked.to_pandas().reset_index().rename(columns={0: 'value'})

In [ ]:
posteriorCheckSims = []

posterior_neuron_inds = []
posterior_spike_times = []
spike_times_lengths = []

for cInd in range(2):
    neuron_inds, spike_times = simulate_brunel(posteriorMaxs_avg[cInd])
    posterior_spike_times.append(spike_times)
    posterior_neuron_inds.append(neuron_inds)
    spike_times_lengths.append(len(spike_times))
    
    checkSim = spike_to_rate(spike_times, remove_tails=False)
    posteriorCheckSims.append(checkSim)

posteriorCheckSims = np.array(posteriorCheckSims).T

In [ ]:
max_spike_number = max(spike_times_lengths)

posterior_check_spikes = xr.DataArray(np.zeros((max_spike_number, 2, 2)), dims=('index', 'source', 'condition'),
                                      coords=dict(source=['times', 'indices'], condition=['Presence','Absence']))

for c_ind in range(2):
    times_length = spike_times_length[c_ind]
    
    posterior_check_spikes[:times_length, 0, c_ind] = posterior_spike_times[c_ind]
    posterior_check_spikes[:times_length, 1, c_ind] = posterior_neuron_inds[c_ind]

    posterior_check_spikes[times_length:, :, c_ind] = np.nan

In [ ]:
plotCheckSims = posteriorCheckSims.copy()
mean_rates = np.array([presence_mean, absence_mean]).T
plotCheckSims = np.array([scale_sim_to_emp(posteriorCheckSims[:,ind].copy(), mean_rates[:,ind]) for ind in range(2)]).T

In [ ]:
fig, ax = plt.subplots(1,1, constrained_layout = True)

alpha = 1

maxLead = 0.5
maxLag = 0.5
timeVec = np.linspace(-maxLead*1000, maxLag*1000, mean_rates.shape[0])
tv = np.linspace(-maxLead*1000, maxLag*1000, posteriorCheckSims.shape[0])

ax.plot(timeVec, mean_rates[:,0], label = 'Emp-Pr', c = 'blue', lw =5, alpha = alpha)
ax.plot(timeVec, mean_rates[:,1], label = 'Emp-Ab', c = 'red', lw = 5, alpha = alpha)

ax.plot(tv, plotCheckSims[:,0].ravel(), label = 'SBI-Pr', lw = 6, c = 'cornflowerblue', ls = ':');
ax.plot(tv, plotCheckSims[:,1].ravel(), label = 'SBI-Ab', lw = 6, c = 'salmon', ls = ':');

ax.set_xlabel('Scaled Time', fontsize = 15, labelpad = 12)
ax.set_ylabel('Normalized Firing Rate', fontsize = 15, labelpad = 10)
ax.legend(fontsize = 12, frameon = False, loc = (0.2, 1.05), ncol = 2)
ax.set_frame_on(False)
ax.tick_params(labelsize = 12, length = 0)
# ax.set_xlim([-500, 500])
fig.tight_layout()
fig.savefig(fig_save_loc + 'SBI_SPK_Timeseries_' + saveString  + '.png', transparent = True, dpi = 400)

In [ ]:
mean_rates = xr.DataArray(mean_rates, dims=('time', 'condition'), coords=dict(condition=['Presence', 'Absence']))
plotCheckSims = xr.DataArray(plotCheckSims, dims=('time', 'condition'), coords=dict(condition=['Presence', 'Absence']))
posterior_array = xr.DataArray(posterior_df_avg['value'].to_numpy(), dims=('sample'), coords=dict(sample=posterior_df_avg['condition'].to_numpy()))

In [ ]:
posterior_data = xr.Dataset(dict(posterior_array=posterior_array, posterior_checks=plotCheckSims, posterior_check_spikes=posterior_check_spikes, emp_rates=mean_rates))
posterior_data.to_netcdf(parent_preprocess_dir + 'Demolliens_SPK_AvgNeurons_Posterior.nc')